# ***Introduction***
Develope by Prof. Dr. Helen Lu, helen.lu@vlerick.com (c) 2026

We use a US SEO dataset to train ML models to predict new equity issuance.

**Models and hyperparameter-tuning:**
*   Logit: no tuning
*   Lasso: no tuning (use default C)
*   Light Gradient Boosting Machine (LightGBM): min_child_samples = max(1% of train, 50); early stopping



**Evaluation:**
Compare  for all three

1.   ROC AUC
2.   Precision/Recall for 10 deciles
1.   Avg predicted prob % vs actual SEO %
1.   Show two-way SHAP plots using winsorized feature values for plotting only (no winsorization in training/prediction)




In [1]:
# Imports libraries
import re
import numpy as np
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, precision_score, recall_score

import lightgbm as lgb
import shap
import matplotlib.pyplot as plt

In [2]:
# Load data and train/validation/test split
DATA_PATH = "https://raw.githubusercontent.com/helenlu-vbs/AI-for-Corporate-Finance-UGent-/refs/heads/main/SEO_teaching.csv"
df = pd.read_csv(DATA_PATH)

def yq_to_int(yq: str) -> int:
    m = re.match(r"(\d{4})q([1-4])", str(yq).lower())
    if m is None:
        raise ValueError(f"Bad yq format: {yq}")
    year, q = int(m.group(1)), int(m.group(2))
    return year * 4 + (q - 1)

df["yq_int"] = df["yq"].apply(yq_to_int)

TEST_START = yq_to_int("2019q3")
TEST_END   = yq_to_int("2020q4")

train = df[df["yq_int"] < TEST_START].copy()
test  = df[(df["yq_int"] >= TEST_START) & (df["yq_int"] <= TEST_END)].copy()

target = "seo_flag"
id_cols = ["gvkey", "yq", "yq_int"]
features = [c for c in df.columns if c not in id_cols + [target]]

X_train = train[features]
y_train = train[target].astype(int).values
X_test  = test[features]
y_test  = test[target].astype(int).values

print("Train:", train.shape, " Pos:", int(y_train.sum()), " Rate:", y_train.mean())
print("Test :", test.shape,  " Pos:", int(y_test.sum()),  " Rate:", y_test.mean())




Train: (15115, 15)  Pos: 111  Rate: 0.007343698312934172
Test : (4219, 15)  Pos: 50  Rate: 0.011851149561507466


In [50]:
# model training
# ------------------------------------------------------------
# Logit
# ------------------------------------------------------------
logit_l2 = Pipeline([
    ("clf", LogisticRegression(
        C=1.0,
        solver="lbfgs",
        max_iter=5000
    ))
])
logit_l2.fit(X_train, y_train)
p_logit = logit_l2.predict_proba(X_test)[:, 1]


# ------------------------------------------------------------
#    LightGBM — only min leaf + early stopping
#    Early stopping needs a validation set. We'll do time-based:
#    last 4 quarters of TRAIN are validation (fallback to last 20% if needed).
# ------------------------------------------------------------
train = train.sort_values("yq_int")
unique_q = np.sort(train["yq_int"].unique())

val_q = unique_q[-4:]               # last 4 quarters as validation
tr_q  = unique_q[:-4]
tr_idx = train["yq_int"].isin(tr_q)
va_idx = train["yq_int"].isin(val_q)

X_tr, y_tr = train.loc[tr_idx, features], train.loc[tr_idx, target].astype(int).values
X_va, y_va = train.loc[va_idx, features], train.loc[va_idx, target].astype(int).values

min_leaf = max(int(0.01 * len(train)), 50)
print("LightGBM min_child_samples =", min_leaf)

lgbm = lgb.LGBMClassifier(
    objective="binary",
    class_weight="balanced",
    random_state=42,
    n_jobs=-1,
    force_col_wise=True,

    # keep parameters simple for class; early stopping will pick best_iteration
    n_estimators=5000,
    learning_rate=0.05,
    num_leaves=31,
    subsample=0.9,
    colsample_bytree=0.9,
    min_child_samples=min_leaf
)

lgbm.fit(
    X_tr, y_tr,
    eval_set=[(X_va, y_va)],
    eval_metric="auc",
    callbacks=[lgb.early_stopping(stopping_rounds=50, verbose=True)]
)

# After early-stopping fit:
best_iter = lgbm.best_iteration_
print("Best iteration from early stopping:", best_iter)

# Combine train + val
X_full = pd.concat([X_tr, X_va], axis=0)
y_full = np.concatenate([y_tr, y_va], axis=0)

# Refit on full (train+val) using best_iter
lgbm_final = lgb.LGBMClassifier(
    objective="binary",
    class_weight="balanced",
    random_state=42,
    n_jobs=-1,
    force_col_wise=True,

    n_estimators=best_iter,      # <-- key line
    learning_rate=0.05,
    num_leaves=31,
    subsample=0.9,
    colsample_bytree=0.9,
    min_child_samples=min_leaf
)

lgbm_final.fit(X_full, y_full)

# Use the refit model for test predictions
p_lgbm = lgbm_final.predict_proba(X_test)[:, 1]


LightGBM min_child_samples = 151
[LightGBM] [Info] Number of positive: 101, number of negative: 11643
[LightGBM] [Info] Total Bins 2576
[LightGBM] [Info] Number of data points in the train set: 11744, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[95]	valid_0's auc: 0.608658	valid_0's binary_logloss: 0.113156
Best iteration from early stopping: 95
[LightGBM] [Info] Number of positive: 111, number of negative: 15004
[LightGBM] [Info] Total Bins 2579
[LightGBM] [Info] Number of data points in the train set: 15115, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000


In [51]:
# Evaluation
# ------------------------------------------------------------
# Compare ROC AUC (test)
# ------------------------------------------------------------
auc_logit = roc_auc_score(y_test, p_logit)
auc_lgbm  = roc_auc_score(y_test, p_lgbm)

print("\n=== Test ROC AUC ===")
print(f"Logit:        {auc_logit:.4f}")
print(f"LightGBM:          {auc_lgbm:.4f}")

# ------------------------------------------------------------
# Precision, recall
# ------------------------------------------------------------




=== Test ROC AUC ===
Logit:        0.5643
LightGBM:          0.7046


In [52]:
# Create a DataFrame for test predictions and actual values
results_df = pd.DataFrame({
    'y_test': y_test,
    'p_logit': p_logit,
    'p_lgbm': p_lgbm
})

# --- Process Logit predictions ---
results_df_logit = results_df.sort_values(by='p_logit', ascending=True).reset_index(drop=True)
results_df_logit['decile'] = pd.qcut(results_df_logit['p_logit'], 10, labels=False, duplicates='drop') + 1

# Calculate actual SEO % for each logit decile
logit_decile_analysis = results_df_logit.groupby('decile')['y_test'].agg(['count', 'sum', 'mean'])
logit_decile_analysis.columns = ['Count', 'Actual_SEO_Count', 'Logit_Actual_SEO_Pct']
logit_decile_analysis['Logit_Actual_SEO_Pct'] = logit_decile_analysis['Logit_Actual_SEO_Pct'] * 100

# --- Process LightGBM predictions ---
results_df_lgbm = results_df.sort_values(by='p_lgbm', ascending=True).reset_index(drop=True)
results_df_lgbm['decile'] = pd.qcut(results_df_lgbm['p_lgbm'], 10, labels=False, duplicates='drop') + 1

# Calculate actual SEO % for each LightGBM decile
lgbm_decile_analysis = results_df_lgbm.groupby('decile')['y_test'].agg(['count', 'sum', 'mean'])
lgbm_decile_analysis.columns = ['Count', 'Actual_SEO_Count', 'LightGBM_Actual_SEO_Pct']
lgbm_decile_analysis['LightGBM_Actual_SEO_Pct'] = lgbm_decile_analysis['LightGBM_Actual_SEO_Pct'] * 100

# Combine results for side-by-side comparison
comparison_df = pd.concat([logit_decile_analysis[['Logit_Actual_SEO_Pct']], lgbm_decile_analysis[['LightGBM_Actual_SEO_Pct']]], axis=1)
comparison_df.index.name = 'Decile'
display(comparison_df.round(2))

,Logit_Actual_SEO_Pct,LightGBM_Actual_SEO_Pct
Decile,,
1,0.47,0.00
2,1.18,0.95
3,0.71,0.71
4,0.95,1.42
5,2.13,0.00
6,1.19,0.71
7,0.95,0.24
8,0.71,1.18
9,2.13,2.61


Local SHAP plot for Amgen Inc. 2020Q4 prediction (Amgen issued shares in 2020Q4)